<a href="https://colab.research.google.com/github/Menaga23hub/GENAI/blob/main/Spam%20detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install required libraries
!pip install gradio pandas scikit-learn nltk

# Import libraries
import gradio as gr
import pandas as pd
import string
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from nltk.corpus import stopwords

# Download NLTK stopwords
nltk.download("stopwords")

# Download and extract SMS Spam dataset
import urllib.request
import zipfile

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip"
zip_path = "smsspamcollection.zip"
urllib.request.urlretrieve(url, zip_path)

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(".")

# Load dataset
df = pd.read_csv("SMSSpamCollection", sep="\t", names=["label", "message"], header=None)
df["label"] = df["label"].map({"ham": 0, "spam": 1})

# Function to clean text
def clean_text(text):
    text = text.lower()  # Lowercase
    text = "".join([char for char in text if char not in string.punctuation])  # Remove punctuation
    text = " ".join([word for word in text.split() if word not in stopwords.words("english")])  # Remove stopwords
    return text

df["clean_message"] = df["message"].apply(clean_text)

# Convert text to numerical features
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df["clean_message"])
y = df["label"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Naive Bayes classifier
model = MultinomialNB()
model.fit(X_train, y_train)

# Function for prediction
def predict_spam(text):
    text_clean = clean_text(text)
    text_vectorized = vectorizer.transform([text_clean])
    prediction = model.predict(text_vectorized)[0]
    return "Spam" if prediction == 1 else "Not Spam"

# Gradio UI
iface = gr.Interface(fn=predict_spam, inputs="text", outputs="text", title="Spam Detector")
iface.launch(share=True)  # Generates a public link

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d96738c836cabeeec8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
